In [ ]:
import pandas as pd
from typing import Optional, List, Union, Tuple
from mbarq.core import Barcode, BarSeqData
from pathlib import Path
import pandas as pd
import sys
#from mbarq.mbarq_logger import get_logger
import subprocess
from Bio.Seq import Seq
from pybedtools import BedTool
import numpy as np
import pyranges as pr

In [ ]:
class AnnotatedMap:
    def __init__(self, map_file: str,
                 annotation_file: str,
                 feature_type: str,
                 identifiers: Tuple[str, ...],
                 name: str = '',
                 output_dir: str = ".",
                 positions: pd.DataFrame = pd.DataFrame()
                 ) -> None:
        self.map_file: Union[str, Path] = map_file
        self.annotations: str = annotation_file
        self.feature_type: str = feature_type
        self.identifiers: Tuple[str, ...] = identifiers
        if name:
            self.name: str = name
        else:
            print(self.map_file)
            self.name = Path(self.map_file).stem
        print(self.name)
        self.output_dir: Path = Path(output_dir)
        self.annotated_map_file: Path = self.output_dir/f'{self.name}.annotated.csv'
        self.positions: pd.DataFrame = positions if not positions.empty else pd.read_csv(self.map_file)
        #self.logger = get_logger('annotate_map-log', self.output_dir / f"{self.name}_annotate_map.log")
        self.temp_bed_file: Path = self.output_dir / f"{self.name}.bed"
        self.temp_bed_results_file: Path = self.output_dir / f"{self.name}.bed.intersect.tab"
        self.temp_bed_closest_file: Path = self.output_dir / f"{self.name}.bed.closest.tab"

    def _find_annotation_overlaps(self, intersect=True):

        """
        Takes output of merge colliding bcs, turns it into bed file, then finds intersections with
        annotation file.
        Generates tab file with the following columns: chr | sstart | gff-info-field | barcode
        """
        #self.logger.info('------------------')
        #self.logger.info('Annotating mapped barcodes')
        bed_map = self.positions.copy().reset_index()
        bed_map['startOffBy1'] = bed_map['insertion_site'] - 1
        bed_map[['chr', 'startOffBy1', 'insertion_site', 'barcode']].to_csv(self.temp_bed_file, sep='\t', index=False,
                                                                            header=False)
        if intersect:
            #self.logger.info('Only annotating barcodes inside/overlapping features of interest')
            command = f"bedtools intersect -wb -b {self.annotations} -a {self.temp_bed_file} " \
                      f" > {self.temp_bed_results_file}"
        else:
            #self.logger.info('Finding closest feature for each barcode')
            tmp_annotations = Path(self.annotations).with_suffix('.sorted.gff')
            tmp_bed = self.temp_bed_file.with_suffix('.sorted.bed')
            command = f"bedtools sort -i {self.temp_bed_file} > {tmp_bed};" \
                      f"bedtools sort -i {self.annotations} | " \
                      f"awk '$3 == \"{self.feature_type}\" {{print $0}} ' > {tmp_annotations};" \
                      f"bedtools closest -b {tmp_annotations} -a {tmp_bed} -D b > {self.temp_bed_results_file}"
        # todo add -id to only display downstream genes
        # todo remove self.temp_bed_closest_file
        #self.logger.info(f'bedtools command: {command}')
        return_code = subprocess.check_call(command, shell=True)
        if return_code != 0:
            #self.logger.error(f"Failed to run bedtools. "
             #                 f"Return code: {return_code}")
            sys.exit(1)
        else:
            pass
            #self.logger.info('Finished finding overlaps between barcodes and features.')
        # todo remove unnecessary files


    def _add_bedtools_results_to_positions(self, intersect=True):
        """
        takes output_map file produced by _find_annotation_overlaps and merges it with barcode map on barcode
        """
        antd_positions = pd.read_table(self.temp_bed_results_file, header=None)
        if intersect:
            antd_positions.columns = ['bc_chr', 'bc_start', 'bc_insertion_site',
                                      'barcode', 'chr', 'source', 'feature', 'feat_start',
                                      'feat_end', 'score', 'strand',
                                      'phase', 'gene_info']
            antd_positions['distance_to_feature'] = 0
        else:
            antd_positions.columns = ['bc_chr', 'bc_start', 'bc_insertion_site',
                                      'barcode', 'chr', 'source', 'feature', 'feat_start',
                                      'feat_end', 'score', 'strand',
                                      'phase', 'gene_info', 'distance_to_feature']
        antd_positions = antd_positions[antd_positions.feature == self.feature_type]
        for feat in self.identifiers:
            pattern = f'({feat}=.+?;|{feat}=.+?$)'
            antd_positions[feat] = (antd_positions['gene_info']
                                    .str.extract(pattern, expand=False)
                                    .str.replace(f'{feat}=', '')
                                    .str.strip(';'))
            if antd_positions[feat].isna().all():
                self.logger.warning(f'"{feat}" identifier not found')
        antd_positions = antd_positions[list(self.identifiers) + ['barcode', 'distance_to_feature']]
        self.annotated_positions = (self.positions.copy()
                                    .merge(antd_positions, how='left', on='barcode'))
        for feat in self.identifiers:
            self.annotated_positions[feat] = (self.annotated_positions[feat]
                                              .fillna(self.annotated_positions['chr'].astype(str) +
                                                      ":" + self.annotated_positions['insertion_site'].astype(str)))
        #self.logger.info(f"writing to {self.annotated_map_file}")
        self.annotated_positions.to_csv(self.annotated_map_file, index=False)
    # os.remove(self.temp_bed_results_file)

    def _validate_annotations(self) -> None:

        """
        Simple validation on the gff file
        only if it's a non-empty file with feature type

        """
        if not Path(self.annotations).is_file():
            #self.logger.error(f'Annotation file "{self.annotations}" not found. Check the path.')
            #self.logger.error(f'To rerun annotation on {self.map_file} run ".... "')
            sys.exit(1)
        else:
            feature_types = pd.read_table(self.annotations, comment='#', usecols=[2]).iloc[:, 0].unique()
            if self.feature_type not in feature_types:
                # todo assumes gff format, might want to generalize later
                #self.logger.error(f'Feature type "{self.feature_type}" not found in {self.annotations}.')
                #self.logger.error(f'Feature types available: {", ".join(feature_types)}')
                sys.exit(1)

    def add_rev_complement(self):
        #self.logger.info(f"Adding barcode reverse complements to {self.annotated_map_file}")
        self.annotated_positions['revcomp_barcode'] = self.annotated_positions.barcode.apply(lambda x: str(Seq(x).reverse_complement()))
        #self.logger.info(f"writing to {self.annotated_map_file}")
        self.annotated_positions.to_csv(self.annotated_map_file, index=False)

    def annotate(self,  intersect=True):
        if self.positions.empty:
            #self.logger.error(f'Found no positions to annotate. Is {self.map_file} empty?')
            sys.exit(1)
        self._validate_annotations()
        self._find_annotation_overlaps(intersect)
        self._add_bedtools_results_to_positions(intersect)

In [ ]:
class AnnotatedMap2:
    def __init__(self, map_file: str,
                 annotation_file: str,
                 feature_type: str,
                 identifiers: Tuple[str, ...],
                 name: str = '',
                 output_dir: str = ".",
                 positions: pd.DataFrame = pd.DataFrame()
                 ) -> None:
        self.map_file: Union[str, Path] = map_file
        self.annotations: str = annotation_file
        self.feature_type: str = feature_type
        self.identifiers: Tuple[str, ...] = identifiers
        if name:
            self.name: str = name
        else:
            print(self.map_file)
            self.name = Path(self.map_file).stem
        print(self.name)
        self.output_dir: Path = Path(output_dir)
        self.annotated_map_file: Path = self.output_dir/f'{self.name}.annotated.csv'
        self.positions: pd.DataFrame = positions if not positions.empty else pd.read_csv(self.map_file)
        #self.logger = get_logger('annotate_map-log', self.output_dir / f"{self.name}_annotate_map.log")
        #self.temp_bed_file: Path = self.output_dir / f"{self.name}.bed"
        #self.temp_bed_results_file: Path = self.output_dir / f"{self.name}.bed.intersect.tab"
        #self.temp_bed_closest_file: Path = self.output_dir / f"{self.name}.bed.closest.tab"

    def _find_annotation_overlaps(self, intersect=True):

        """

        """
        #self.logger.info('------------------')
        #self.logger.info('Annotating mapped barcodes')
        bed_map = self.positions[['chr', 'insertion_site', 'barcode', 'number_of_reads']].copy()
        bed_map['startOffBy1'] = bed_map['insertion_site'] - 1
        bed_map = bed_map[['chr',  'startOffBy1', 'insertion_site',  'barcode', 'number_of_reads']]
        barcodes_bedtool = BedTool.from_dataframe(bed_map).sort()
        
        gff = pr.read_gff3(self.annotations)
        gff.Start = gff.Start + 1
        to_keep = ['Chromosome', 'Source', 'Feature', 'Start', 'End', 'Score', 'Strand'] + self.identifiers
        gff_short = gff[gff.Feature == self.feature_type][to_keep].as_df()
        genes_bedtool = BedTool.from_dataframe(gff_short).sort()
        names = ['bc_chr', 'bc_start', 'insertion_site', 'barcode', 'abundance_in_mapping_library', 
                 'chr', 'source', 'feature', 'gene_start', 'gene_end', 
                 'score', 'gene_strand'] + self.identifiers + ['distance_to_feature']
        barcode_annotation = barcodes_bedtool.closest(genes_bedtool, D='b').to_dataframe(names=names)

        final_columns = ['barcode', 'chr', 'insertion_site', 'abundance_in_mapping_library',
                        'gene_start', 'gene_end', 'gene_strand'] + self.identifiers + ['distance_to_feature']
        if intersect == True:
            barcode_annotation = barcode_annotation[barcode_annotation['distance_to_feature'] == 0]
        #calculate percentile
        barcode_annotation['percentile'] = np.where(barcode_annotation.gene_strand.isin('+', 'plus'), 
                                          barcode_annotation.gene_start, intersect_df.Start_b, intersect_df.End_b), 
                                          min_max(intersect_df.Start, intersect_df.End_b, intersect_df.Start_b))
        self.annotated_positions = barcode_annotation[final_columns]
        self.annotated_positions.to_csv(self.annotated_map_file, index=False)
        

    def _validate_annotations(self) -> None:

        """
        Simple validation on the gff file
        only if it's a non-empty file with feature type

        """
        if not Path(self.annotations).is_file():
            #self.logger.error(f'Annotation file "{self.annotations}" not found. Check the path.')
            #self.logger.error(f'To rerun annotation on {self.map_file} run ".... "')
            sys.exit(1)
        else:
            feature_types = pd.read_table(self.annotations, comment='#', usecols=[2]).iloc[:, 0].unique()
            if self.feature_type not in feature_types:
                # todo assumes gff format, might want to generalize later
                #self.logger.error(f'Feature type "{self.feature_type}" not found in {self.annotations}.')
                #self.logger.error(f'Feature types available: {", ".join(feature_types)}')
                sys.exit(1)

    def add_rev_complement(self):
        #self.logger.info(f"Adding barcode reverse complements to {self.annotated_map_file}")
        self.annotated_positions['revcomp_barcode'] = self.annotated_positions.barcode.apply(lambda x: str(Seq(x).reverse_complement()))
        #self.logger.info(f"writing to {self.annotated_map_file}")
        self.annotated_positions.to_csv(self.annotated_map_file, index=False)

    def annotate(self,  intersect=True):
        if self.positions.empty:
            #self.logger.error(f'Found no positions to annotate. Is {self.map_file} empty?')
            sys.exit(1)
        self._validate_annotations()
        self._find_annotation_overlaps(intersect)
        
        

In [ ]:
root = Path("/nfs/cds-peta/exports/biol_micro_cds_gr_sunagawa/scratch/Projects_NCCR/ref/mbarq_test_data")

In [ ]:
map_file = root/'dnaid1315/expected_outcomes/23-06-22-library_11_1.map.csv'
gff_file = root/"dnaid1315/ref/GCA_000210855.2_ASM21085v2_genomic.gff"
feature_type = 'gene'
identifiers = ['Name', 'locus_tag']

In [ ]:
am = AnnotatedMap(map_file, gff_file, feature_type, identifiers)
am2 = AnnotatedMap2(map_file, gff_file, feature_type, identifiers)

In [ ]:
am.annotate()
am2.annotate()

In [ ]:
eo = am.annotated_positions.copy()
no = am2.annotated_positions.copy()
no = (no[['barcode', 'insertion_site', 'Name', 'locus_tag', 'distance_to_feature' ]]
    .sort_values(['barcode', 'locus_tag'])
    .reset_index()
     .drop(['index'], axis=1))
eo = (eo[['barcode', 'insertion_site', 'Name', 'locus_tag', 'distance_to_feature' ]]
      .sort_values(['barcode', 'locus_tag'])
      .reset_index()
      .drop(['index'], axis=1))
#eo['Gene_strand'] = eo['Gene_strand'].replace({'minus':'+', 'plus':'-'})

In [ ]:
eo.head()

In [ ]:
eo.head(80).tail(10)

In [ ]:
eo[eo.locus_tag.str.contains(':')]

In [ ]:
eo.shape

In [ ]:
no.shape

In [ ]:
eo.head(80).tail(10).equals(no.head(80).tail(10))

In [ ]:
enull = eo[eo.distance_to_feature == 0]

In [ ]:
nnull = no[no.distance_to_feature == 0]

In [ ]:
eo[eo.barcode == 'CCACTCCGCTTCTTGCA']

In [ ]:
no[no.barcode == 'GCGCATCATTGCATGCA']

In [ ]:
gff = pr.read_gff3(gff_file)

In [ ]:
2884860 - 2884712

In [ ]:
2885006 - 2884860

In [ ]:
gff.head()

In [ ]:
gff[gff.ID == 'gene-SL1344_2704']

In [ ]:
enull.merge(nnull, how='outer', on='barcode')

In [ ]:
eo[eo.barcode == 'GGATAGCACCCAAAAAG']

In [ ]:
no[no.barcode == 'GGATAGCACCCAAAAAG']

In [ ]:
1551266-1551220

In [ ]:
am.annotated_positions.merge(am2.annotated_positions, on='barcode').sample(20)

In [ ]:
eo[eo.distance_to_feature != 0].head(20)

In [ ]:
no.head()

In [ ]:
expected_outcome = expected_outcome [['eo_barcode', 'eo_insertion_site', 'eo_Name', 'eo_locus_tag', 'eo_distance_to_feature']]

In [ ]:
#map_file = root/'dnaid1315/expected_outcomes/23-06-22-library_11_1.map.csv'
#bed_file = root/'dnaid1315/expected_outcomes/library_11_1.bed'
gff_file = root/"dnaid1315/ref/GCA_000210855.2_ASM21085v2_genomic.gff"
feature_type = 'gene'
identifiers = ['Name', 'locus_tag']

In [ ]:
bc_df = am.positions[['chr', 'insertion_site', 'barcode', 'number_of_reads']]
bc_df['Start'] = bc_df['insertion_site'] - 1
bc_df = bc_df[['chr', 'Start', 'insertion_site', 'barcode', 'number_of_reads']]
barcodes = BedTool.from_dataframe(bc_df).sort()
gff = pr.read_gff3(gff_file)
to_keep = ['Chromosome', 'Source', 'Feature', 'Start', 'End', 'Score', 'Strand'] + identifiers
anot = gff[gff.Feature == feature_type][to_keep].as_df()
genes = BedTool.from_dataframe(anot).sort()

In [ ]:
names = ['bc_chr', 'bc_start', 'bc_insertion_site',
                                      'barcode', 'number_of_reads', 'chr', 'source', 'feature', 'feat_start',
                                      'feat_end', 'score', 'strand'] + identifiers + ['distance_to_feat']
                                      

In [ ]:
bc_df

In [ ]:
cl = barcodes.closest(genes, D='b').to_dataframe(names=names)

In [ ]:
cl

In [ ]:
4703859 - 4703820

In [ ]:
cl[cl.barcode == 'ACTGTTGCGACACTTCA']

In [ ]:
eo[eo.barcode == 'TGACCACGTTAGCATAG']

In [ ]:
fdf[fdf.barcode == 'GGATAGCACCCAAAAAG']

In [ ]:
fdf.groupby('barcode').eo_barcode.count().reset_index().sort_values('eo_barcode').tail(10)

In [ ]:
g_and_b = genes.intersect(barcodes, wb=True).to_dataframe()

In [ ]:
expected_outcome.sort_values('barcode')

In [ ]:
b_and_g.sort_values('name')

In [ ]:


def _find_annotation_overlaps(map_file, gff_file, feature_type, identifiers, intersect=True):
    
    def min_max(isite, start, end):
        return round((isite - start)/(end-start),2)
    
    gff = pr.read_gff3(gff_file)
    to_keep = ['Chromosome', 'Start', 'End', 'Strand'] + identifiers
    anot = gff[gff.Feature == feature_type][to_keep]
    positions = pd.read_csv(map_file)
    pos = positions.rename({'chr': 'Chromosome', 'insertion_site': 'Start'}, axis=1).drop('strand', axis=1)
    pos['End'] = pos['Start']
    pos = pr.PyRanges(pos)
    df = pos.k_nearest(anot, k=3, apply_strand_suffix=False).as_df()
    df['Distance'] = abs(df['Distance'])
    min_df = df.groupby('barcode').Distance.min().reset_index()
    final = min_df.merge(df, on=['barcode', 'Distance'], how='left')
    intersect_df = final[final.Distance == 0].copy()
    intersect_df['percentile'] = np.where(intersect_df.Strand == '+', 
                                          min_max(intersect_df.Start, intersect_df.Start_b, intersect_df.End_b), 
                                          min_max(intersect_df.Start, intersect_df.End_b, intersect_df.Start_b))
    if intersect == True:
        return intersect_df
    else:
        closest_df = final[final.Distance != 0].copy()
        print(closest_df[closest_df.barcode == 'GGAGTGTCCGTAGGCTG'])
        closest_df['in_front'] = np.where(closest_df.Strand == '+', 
                                          closest_df.Start < closest_df.Start_b,
                                          closest_df.Start > closest_df.End_b)
        closest_df['in_front'] = closest_df.in_front.replace({True: -1, False: 1})
        print(closest_df[closest_df.barcode == 'GGAGTGTCCGTAGGCTG'])
        closest_df['Distance'] = closest_df.in_front * closest_df.Distance
        print(closest_df[closest_df.barcode == 'GGAGTGTCCGTAGGCTG'])  
        closest_df = closest_df.drop(['in_front'], axis=1)
        print(closest_df[closest_df.barcode == 'GGAGTGTCCGTAGGCTG'])
        return pd.concat([intersect_df, closest_df])
    

In [ ]:
# todo write validation code
# Assume Feature and identifiers will be in columns

In [ ]:
fdf = _find_annotation_overlaps(map_file, gff_file, feature_type, identifiers, intersect=False)

In [ ]:
fdf[fdf.barcode == 'AAAAAGGACAATATGCG']

In [ ]:
expected_outcome[expected_outcome.barcode == 'AAAAAGGACAATATGCG']

In [ ]:
fdf.groupby('barcode').locus_tag.count().reset_index().sort_values('locus_tag').tail(10)

In [ ]:
fdf[fdf.barcode == 'GGATAGCACCCAAAAAG']

In [ ]:
am.annotated_positions.groupby('barcode').locus_tag.count().reset_index().sort_values('locus_tag').tail(10)

In [ ]:
fdf = 

In [ ]:
fdf.barcode.nunique()

In [ ]:
test = pos[pos.barcode ==  'TCCACTGGAAGGAGAGC']

In [ ]:
test

In [ ]:
x = pos.intersect(anot)#.as_df().merge(pos.as_df(), on=['Chromosome', 'Start', 'End'])

In [ ]:
pos.barcode.nunique()

In [ ]:
gr1 = pr.from_dict({'Chromosome': ['FQ312003.1'] , 
                    'Start':[878889], 'End': [878890], 'ID': ['TCCACTGGAAGGAGAGC']})
gr2 = pr.from_dict({'Chromosome': ['FQ312003.1', 'FQ312003.1'],
                   'Start': [878495, 878867], 
                   'End': [878906, 879974],
                   'Name': ['ybhQ', 'ybhR']})


In [ ]:
gr1.nearest(gr2)

In [ ]:
gr1.k_nearest(gr2, k=1)

In [ ]:
gr1.k_nearest(gr2, k=2)

In [ ]:
gr1.k_nearest(gr2, k=1, strandedness=None, apply_strand_suffix=True)

In [ ]:
gr1.k_nearest(gr2, k=1, strandedness=None, apply_strand_suffix=False)

In [ ]:
test

In [ ]:
test.k_nearest(anot, k=1)

In [ ]:
gr1.k_nearest(gr2, k=1, apply_strand_suffix=True)

In [ ]:
gr1.nearest(gr2, apply_strand_suffix=False)

In [ ]:
?pos.nearest

In [ ]:
df = pos.k_nearest(anot, k=2,strandedness=False, apply_strand_suffix=False).as_df()

In [ ]:
df[df.barcode == "AAAAAGGACAATATGCG"]

In [ ]:
df.Distance.hist()

In [ ]:
df2.Distance.hist()

In [ ]:
df.head()

In [ ]:
max_df  = df.groupby('barcode').Distance.max().reset_index()
max_df[max_df.Distance <= 0]

In [ ]:
min_df  = df.groupby('barcode').Distance.min().reset_index()

In [ ]:
l = df.merge(min_df, on=['barcode', 'Distance'], how='inner')

In [ ]:
min_df

In [ ]:
df

In [ ]:
l

In [ ]:
x.barcode.nunique()

In [ ]:
fdf[fdf.barcode == 'TCCACTGGAAGGAGAGC']

In [ ]:
expected_outcome[expected_outcome.barcode == 'TCCACTGGAAGGAGAGC']

In [ ]:
expected_outcome[expected_outcome.barcode.duplicated()]

In [ ]:
p0[p0.barcode == 'GTACTACGATTCGGAAC']

In [ ]:
expected_outcome.dropna(subset=['distance_to_feature']).head(10)

In [ ]:
p0.to_df().sort_values('barcode')

In [ ]:
t  = p0[p0.Strand_b =='+'] 

min_max(t.Start, t.Start_b, t.End_b)

In [ ]:
p0.apply(min_max)

In [ ]:
(16097-16087)/(16432 - 16087)

In [ ]:
10/345*100

In [ ]:
(21143-20057)/(23054- 20057)

In [ ]:
23054- 20057

In [ ]:
21143-20057